In [ ]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from matplotlib import rc
from numpy import inf

from ROOT import TLatex

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

In [ ]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

plot_style_3 = {
    'histtype': 'step',
    'linewidth': 2,
    'density': True,
    'alpha':0.8
}

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

In [ ]:
def applyCuts(df,isMC=False):
    temp = df
    temp.eval('theta_lab = arccos(Pz/P)*180.0/3.14159',inplace=True)
    temp.eval('inelasticity = Nu/5.014', inplace=True)
    temp.eval('E = sqrt(P*P+0.1396*0.1396)', inplace=True)
    if(isMC):
        print ('MC')
    else:
        print ('Data')
        temp.eval('pass_reco=1',inplace=True)

    temp = applyCut(temp, 'TargType==1','TargetType==1')
    temp = applyCut(temp, 'pass_reco==0 | inelasticity<0.85','inelasticity < 0.85')
    temp = applyCut(temp, 'pass_reco==0 | (theta_lab<120 and theta_lab>10)', '10<theta_lab<120')
    temp = applyCut(temp, 'pass_reco==0 | (P>0.200)', 'P>0.200')
    temp = applyCut(temp, 'pass_reco==0 | abs(deltaZ)<3', 'abs(deltaZ)<3 [cm]')

    temp = applyCut(temp,'pass_reco==0 |(PID==211)| (PID==-211 & theta_lab>30)| (PID==-211 & theta_lab<30 & P>0.600)','(PID==211)| (PID==-211 & theta_lab>30)| (PID==-211 & theta_lab<30 & P>0.600)')
    print('################\n')
    return temp

In [ ]:
path = '/home/miguel/EG2_DATA/sebastian/'
data_hadron = root_pandas.read_root(path+'C.root', "ntuple_data")

## obtain MC

In [ ]:
path = '/home/miguel/EG2_DATA/andres/'
mc_hadron = root_pandas.read_root(path+'D_weighted.root', "ntuple_sim")

In [ ]:
data_hadron['z'] = data_hadron['Zh']


## Apply cuts

In [ ]:
data= applyCuts(data_hadron, isMC=False)
mc= applyCuts(mc_hadron, isMC=True)

## Delta-z

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(data_hadron.query('PID==211')['deltaZ'],bins=40,range=(-10,10), **plot_style_3,label='$\pi^{+}$')
plt.hist(data_hadron.query('PID==-211')['deltaZ'],bins=40,range=(-10,10), **plot_style_3,label='$\pi^{-}$')
plt.ylabel('Normalized entries')
plt.xlabel('$\Delta$ Z [cm]')
plt.legend(loc='best',frameon=False)
plt.axvline(x=3.0,ls='--',color='black')
plt.axvline(x=-3.0,ls='--',color='black')

#plt.yscale('log')
plt.tight_layout()
plt.savefig('DeltaZ_distribution.pdf')
plt.show()

In [ ]:
plt.hist(data.query('PID==211')['deltaZ'],bins=100,alpha=0.3,range=(-5,5))
plt.hist(data.query('PID==-211')['deltaZ'],bins=100,alpha=0.3,range=(-5,5))

plt.show()

## pT vs z 

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-180, 180, 25)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['PhiPQ'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['PhiPQ'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc weighted')
    axs[j].hist(data.query(query)['PhiPQ'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$\phi_{PQ}$',fontsize=25)

#axs[j].set_yscale('log')

    print(query)
axs[0].legend(frameon=False,loc='best')


In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(24,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0, 2.0, 20)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Pt2'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc weighted')
    axs[j].hist(data.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{T}^{2}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True,figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-1.8, 1.8, 25)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Px'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Px'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w.')
    axs[j].hist(data.query(query)['Px'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{x}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-1.5, 1.5, 25)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Py'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Py'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['Py'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{y}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0.0, 4.5, 20)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['P'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['P'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['P'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(-3.0,3.0 , 40)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['deltaZ'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['deltaZ'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['deltaZ'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=25)
    axs[j].set_xlabel(r'$\Delta$ z',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')



In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4),gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(1.0,4.0 , 40)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Q2'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Q2'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['Q2'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$Q^{2}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(24,4),gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(2.0,4.5 , 20)

    query = 'z>%2.2f and z<%2.2f and PID==211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Nu'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Nu'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['Nu'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$\nu$',fontsize=22)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')


## pi-

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, figsize=(12,6),gridspec_kw={'hspace': 0, 'wspace':0})

binning = np.linspace(0.0, 1.0, 50)
query = 'PID==211'
axs[0].hist(mc.query(query)['z'],density=True,bins=binning,alpha=0.5,label='mc')
axs[0].hist(mc.query(query)['z'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
axs[0].hist(data.query(query)['z'],density=True,bins=binning,alpha=0.5,label='data')
#axs[0].set_yscale('log')
axs[0].set_title(r'$\pi^{+}$',fontsize=25)

query = 'PID==-211'
axs[1].hist(mc.query(query)['z'],density=True,bins=binning,alpha=0.5,label='mc')
axs[1].hist(mc.query(query)['z'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
axs[1].hist(data.query(query)['z'],density=True,bins=binning,alpha=0.5,label='data')
#axs[1].set_yscale('log')
axs[1].set_title(r'$\pi^{-}$',fontsize=25)
axs[0].set_xlabel(r'$z$',fontsize=25)

axs[1].set_xlabel(r'$z$',fontsize=25)
axs[0].set_ylabel('Normalized entries',fontsize=25)

axs[0].legend(frameon=False,loc='best')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, figsize=(24,4))


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0.0, 4.5, 25)

    query = 'z>%2.2f and z<%2.2f and PID==-211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['P'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['P'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc w')
    axs[j].hist(data.query(query)['P'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
#axs[0].set_yscale('log')

In [ ]:
fig, axs = plt.subplots(1, 5, sharey=True, sharex=True, figsize=(24,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = [0.0,0.2,0.4,0.6,0.8,1.0]
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

for j, ibin in enumerate(bins):
    binning = np.linspace(0, 2.0, 20)

    query = 'z>%2.2f and z<%2.2f and PID==-211'%(ibin[0],ibin[1])
    axs[j].hist(mc.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='mc')
    axs[j].hist(mc.query(query)['Pt2'],bins=binning,weights = mc.query(query)['weight_reco'],**plot_style_1,label='mc weighted')
    axs[j].hist(data.query(query)['Pt2'],density=True,bins=binning,alpha=0.5,label='data')

    axs[j].set_title('%2.2f<z<%2.2f'%(ibin[0],ibin[1]),fontsize=20)
    axs[j].set_xlabel(r'$P_{T}^{2}$',fontsize=25)


    print(query)
axs[0].legend(frameon=False,loc='best')
axs[0].set_yscale('log')

In [ ]:
## Compute efficiency 

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(16,5),sharey=True)

mask = 'pass_reco==1'

bins = np.linspace(0,1.0,40)
axs[0].set_title('Non weighted')
axs[1].set_title('Weighted')
axs[0].hist(mc_hadron['mc_z'],alpha=0.3,bins=bins, label='all generated')
axs[0].hist(mc_hadron.query('pass_reco==1')['mc_z'], alpha=0.3,bins=bins, label='pass reco')

axs[1].hist(mc_hadron.query('pass_truth==1')['mc_z'],alpha=0.3,bins=bins,weights=mc_hadron.query('pass_truth==1')['weight_truth'],label='all generated')
axs[1].hist(mc_hadron.query(mask)['mc_z'],alpha=0.3,bins=bins,weights=mc_hadron.query(mask)['weight_truth'],label='pass reco')


axs[0].set(xlabel=r"$z$", ylabel="Events")
axs[1].set(xlabel=r"$z$", ylabel="Events")
plt.legend()
plt.show()

In [ ]:
print('Pass reco ' , np.count_nonzero(mc_hadron['pass_reco']))
print('Pass truth ', np.count_nonzero(mc_hadron['pass_truth']))

print(len(mc_hadron.query('pass_reco==1')))
print(len(mc_hadron))

In [ ]:
for i in mc_hadron.keys():
    print(i)